## Round 2

In [17]:
# ! cp /media/DATA/gwasim/magma_outputs/random_top/* /media/DATA/gwasim/round2/bioGWAS/tests/3_pathways/data_enrich

In [41]:
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm
from collections import defaultdict
import os

PATH_OUT_DATA = "/media/DATA/gwasim/round2/bioGWAS/tests/3_pathways/data_enrich"
pathes = {
    "path_big": "KEGG_FOCAL_ADHESION",
    "path_medium": "KEGG_PPAR_SIGNALING_PATHWAY",
    "path_small": "KEGG_STEROID_BIOSYNTHESIS",
    "path_random": "XXXXX",
}
PATHES_ORDER = ["path_big", 
                "path_medium", 
                "path_small", 
                "path_random"
               ]

models = {
    "linreg": "linreg",
    "mean": "snp-wise=mean",
    "top": "snp-wise=top",
}

MODELS_TO_DRAW=['linreg', 
                'mean', 
                'top', 
                'PASCAL',
               ]

ALPHA = 0.05
COL_P = "P_adj"
ITER = 30


def return_stats(file_name, pathname):
    genesets = []

    try:
        with open(file_name, "r") as file:
            for line in file:
                # Check if the line starts with the desired pattern
                if line.startswith("# _SET") and "VARIABLE" in line:
                    # Extracting the part after equality sign and before (set)
                    part = line.split("=")[-1].split("(set)")[0].strip()
                    genesets.append(part)
    except FileNotFoundError:
        pass
    check = {
        "avg#": len(genesets),
        "FPR": int((len(genesets) > 0) and (pathname not in genesets)),
        "TPR": int(pathname in genesets),
    }
    return check


def return_stats_pascal(data, pathname, ALPHA):
    genesets = set(data[data[COL_P]<ALPHA].Name)
#     print(pathname, genesets)
    check = {
        "avg#": len(genesets),
        "FPR": int((len(genesets) > 0) and (pathname not in genesets)),
        "TPR": int(pathname in genesets),
    }
    return check


def aggregate_summaries(sum1):
    grouped_mean = sum1.groupby(level=0).mean()
    # Group by level 0 and calculate the size of each group
    grouped_size = sum1.groupby(level=0).size()
    grouped_size = grouped_size.reset_index(name="size")
    # Merge the calculated mean and size DataFrames
    aggsum1 = (
        grouped_mean.reset_index().merge(grouped_size, on="index").set_index("index")
    )
    aggsum1.index = pd.CategoricalIndex(
        aggsum1.index, categories=PATHES_ORDER, ordered=True
    )
    aggsum1 = aggsum1.sort_index()
    return aggsum1

def get_confidences(sum1):
    confidences = defaultdict(dict)
    for pathname in PATHES_ORDER:
        filtered_sum = sum1[sum1.index.isin([pathname], level=0)]
        for col in ['FPR', 'TPR']:
            confidences[pathname][f"{col}_confidence"] = proportion_confint(filtered_sum[col].sum(),filtered_sum.shape[0], method='wilson')
    aggsum1_conf = pd.DataFrame(confidences).T
    return aggsum1_conf

## Aggregate MAGMA results

In [42]:
results_TPR = dict()
results_FPR = dict()
for model in models:
    print(f"MODEL: {model}")

    summary_table1 = dict()

    for pathfile in pathes:
        pathname = pathes[pathfile]
        for i in range(ITER):
            cas_id = f"{pathfile}_{i}"
            SIM_ID = f"test10000_{cas_id}_{cas_id}"
            GENE_SETS_OUT = f"{PATH_OUT_DATA}/{SIM_ID}_magma_sets_{model}"
            method1_file = f"{GENE_SETS_OUT}.gsa.sets.genes.out"
            cur_res  = return_stats(method1_file, pathname)
            if os.path.isfile(method1_file.replace('.sets.genes.out', '.out')):
                summary_table1[(pathfile, i)] = cur_res
#     print(f'Col, used as p-value: {COL_P}\nSignifigance level: {ALPHA}')
    sum1 = pd.DataFrame(summary_table1).T
    aggsum1 = aggregate_summaries(sum1)
    aggsum1_conf = get_confidences(sum1)
    aggregated = pd.merge(aggsum1, aggsum1_conf, left_index=True, right_index=True)
#     print(f'Magma {models[model]} results:')
    display(aggregated)
    print('============================================================================\n')
    for path in ["path_small", "path_medium", "path_big"]:
        results_TPR[(model, path)] = aggregated.TPR[path], *aggregated.TPR_confidence[path]
    for path in ["path_random",]:
        if path not in pathes:
            continue
        results_FPR[(model, path)] = aggregated.FPR[path], *aggregated.FPR_confidence[path]

MODEL: linreg


,avg#,FPR,TPR,size,FPR_confidence,TPR_confidence
path_big,1.000000,0.233333,0.300000,30,"(0.11792388144489496, 0.40928326158122175)","(0.16664748268243793, 0.4787578745871496)"
path_medium,1.066667,0.133333,0.633333,30,"(0.05309655484054743, 0.296813266820363)","(0.45513563654794864, 0.7812607919389929)"
path_small,1.133333,0.000000,1.000000,30,"(0.0, 0.1135133931739688)","(0.8864866068260311, 1.0)"
path_random,0.000000,0.000000,0.000000,30,"(0.0, 0.1135133931739688)","(0.0, 0.1135133931739688)"



MODEL: mean


,avg#,FPR,TPR,size,FPR_confidence,TPR_confidence
path_big,0.233333,0.166667,0.066667,30,"(0.07336542371848548, 0.33564350506416035)","(0.018477023791270378, 0.21323458362616926)"
path_medium,0.500000,0.100000,0.333333,30,"(0.03459988874733416, 0.25621082579184085)","(0.19230498083676129, 0.5121994835545616)"
path_small,1.200000,0.033333,0.966667,30,"(0.005908590381612455, 0.16670390991409176)","(0.8332960900859081, 0.9940914096183875)"
path_random,0.100000,0.100000,0.000000,30,"(0.03459988874733416, 0.25621082579184085)","(0.0, 0.1135133931739688)"



MODEL: top


,avg#,FPR,TPR,size,FPR_confidence,TPR_confidence
path_big,2.000000,0.433333,0.300000,30,"(0.2737748557646652, 0.608026929991864)","(0.16664748268243793, 0.4787578745871496)"
path_medium,2.066667,0.233333,0.566667,30,"(0.11792388144489496, 0.40928326158122175)","(0.39197307000813597, 0.7262251442353348)"
path_small,2.600000,0.000000,1.000000,30,"(0.0, 0.1135133931739688)","(0.8864866068260311, 1.0)"
path_random,0.566667,0.433333,0.000000,30,"(0.2737748557646652, 0.608026929991864)","(0.0, 0.1135133931739688)"


## Aggregate Pascal results

In [43]:
model='PASCAL'
# results_TPR = dict()
# results_FPR = dict()

summary_table1 = dict()
for pathname in pathes:
    for i in range(ITER):
        
        index=f"{pathname}_{i}"
        file_name = f"/media/DATA/gwasim/pascal_outputs/test10000_{index}_{index}_gwas_rsid.PathwaySet--msigBIOCARTA_KEGG_REACTOME--sum.txt"

        data = pd.read_csv(file_name, sep='\t')
        # filter only KEGS
        data = data[data.Name.apply(lambda x: 'KEGG' in  x)]
        
        data['P_adj'] = fdrcorrection(data.empPvalue)[1]
        cur_res  = return_stats_pascal(data, pathes[pathname], ALPHA)
        summary_table1[(pathname, i)] = cur_res
        
sum1 = pd.DataFrame(summary_table1).T
aggsum1 = aggregate_summaries(sum1)
aggsum1_conf = get_confidences(sum1)
aggregated = pd.merge(aggsum1, aggsum1_conf, left_index=True, right_index=True)
display(aggregated)

for path in ["path_small", "path_medium", "path_big"]:
    results_TPR[(model, path)] = aggregated.TPR[path], *aggregated.TPR_confidence[path]
for path in ["path_random"]:
    if path not in pathes:
        continue
    results_FPR[(model, path)] = aggregated.FPR[path], *aggregated.FPR_confidence[path]

,avg#,FPR,TPR,size,FPR_confidence,TPR_confidence
path_big,3.600000,0.033333,0.5,30,"(0.005908590381612455, 0.16670390991409176)","(0.3315412564053376, 0.6684587435946624)"
path_medium,0.900000,0.033333,0.6,30,"(0.005908590381612455, 0.16670390991409176)","(0.42320360253322475, 0.7540937188319815)"
path_small,0.866667,0.033333,0.8,30,"(0.005908590381612455, 0.16670390991409176)","(0.6269430358685175, 0.9049489282271013)"
path_random,0.066667,0.066667,0.0,30,"(0.018477023791270378, 0.21323458362616926)","(0.0, 0.1135133931739688)"


In [44]:
results_TPR

{('linreg', 'path_small'): (1.0, 0.8864866068260311, 1.0),
 ('linreg', 'path_medium'): (0.6333333333333333,
  0.45513563654794864,
  0.7812607919389929),
 ('linreg', 'path_big'): (0.3, 0.16664748268243793, 0.4787578745871496),
 ('mean', 'path_small'): (0.9666666666666667,
  0.8332960900859081,
  0.9940914096183875),
 ('mean', 'path_medium'): (0.3333333333333333,
  0.19230498083676129,
  0.5121994835545616),
 ('mean', 'path_big'): (0.06666666666666667,
  0.018477023791270378,
  0.21323458362616926),
 ('top', 'path_small'): (1.0, 0.8864866068260311, 1.0),
 ('top', 'path_medium'): (0.5666666666666667,
  0.39197307000813597,
  0.7262251442353348),
 ('top', 'path_big'): (0.3, 0.16664748268243793, 0.4787578745871496),
 ('PASCAL', 'path_small'): (0.8, 0.6269430358685175, 0.9049489282271013),
 ('PASCAL', 'path_medium'): (0.6, 0.42320360253322475, 0.7540937188319815),
 ('PASCAL', 'path_big'): (0.5, 0.3315412564053376, 0.6684587435946624)}

## All together + save

In [45]:
data_TPR = pd.DataFrame(results_TPR).T.rename(columns={0:'score', 1:'min', 2:'max'})
data_FPR = pd.DataFrame(results_FPR).T.rename(columns={0:'score', 1:'min', 2:'max'})
data_TPR.index = data_TPR.index.set_names(['model', 'path'])
data_FPR.index = data_FPR.index.set_names(['model', 'path'])
data_TPR = data_TPR[data_TPR.index.isin(MODELS_TO_DRAW, level=0)]
data_FPR = data_FPR[data_FPR.index.isin(MODELS_TO_DRAW, level=0)]

display(data_TPR)
display(data_FPR)

score       min       max
model  path                                     
linreg path_small   1.000000  0.886487  1.000000
       path_medium  0.633333  0.455136  0.781261
       path_big     0.300000  0.166647  0.478758
mean   path_small   0.966667  0.833296  0.994091
       path_medium  0.333333  0.192305  0.512199
       path_big     0.066667  0.018477  0.213235
top    path_small   1.000000  0.886487  1.000000
       path_medium  0.566667  0.391973  0.726225
       path_big     0.300000  0.166647  0.478758
PASCAL path_small   0.800000  0.626943  0.904949
       path_medium  0.600000  0.423204  0.754094
       path_big     0.500000  0.331541  0.668459

,,score,min,max
model,path,,,
linreg,path_random,0.000000,0.000000,0.113513
mean,path_random,0.100000,0.034600,0.256211
top,path_random,0.433333,0.273775,0.608027
PASCAL,path_random,0.066667,0.018477,0.213235


In [46]:
data_TPR.to_csv("data_enrich/TPR_to_draw.csv")
data_FPR.to_csv("data_enrich/FPR_to_draw.csv")